In [1]:
import numpy as np
import torch

# 1. Создание тензора
t_from_list = torch.tensor([[1, 2], [3, 4]])  # копия
t_from_np = torch.from_numpy(np.array([[5., 6.], [7., 8.]]))  # shared memory

# 2. reshape, transpose, slicing
x = torch.arange(12)  # tensor([0,1,2,...,11])
a = x.reshape(3, 4)  # 3×4
b = a.transpose(0, 1)  # или a.t() для 2D
c = a.permute(1, 0)  # то же, что transpose

print(a[1:, 2:])  # tensor([[ 6, 7],[10,11]])
print(a[:, ::2])  # каждый второй столбец

# 3. GPU (если есть)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_gpu = x.to(device)  # или .cuda() если точно есть GPU
# model = model.to(device)
# data = data.to(device)

# 4. PyTorch vs NumPy — когда что
# NumPy          → анализ данных, pandas, маленькие массивы, CPU-only, нет градиентов
# PyTorch        → нейросети, GPU, autograd, большие данные, обучение


# TODO:
# 1. Создать tensor из list, numpy array
# 2. Операции: reshape, transpose, slicing
# 3. GPU transfer (если есть GPU, иначе просто изучить синтаксис)
# 4. Сравнить с NumPy (когда что использовать)

tensor([[ 6,  7],
        [10, 11]])
tensor([[ 0,  2],
        [ 4,  6],
        [ 8, 10]])


In [2]:
import torch

# 1. Создаём тензор, который "следит" за градиентами
x = torch.tensor([2.0], requires_grad=True)
# или короче: x = torch.tensor(2.0, requires_grad=True)

print(x)  # tensor([2.], requires_grad=True)

# 2. Простая операция → строится вычислительный граф
y = x ** 2  # y = x²

print(y)  # tensor([4.], grad_fn=<PowBackward0>)

# 3. Запускаем обратное распространение от y
y.backward()  # вычисляет ∂y/∂x для всех листьев графа

# 4. Градиент теперь лежит в .grad
print(x.grad)  # tensor([4.])

tensor([2.], requires_grad=True)
tensor([4.], grad_fn=<PowBackward0>)
tensor([4.])


In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import requests
from io import BytesIO

# 1. Загрузка предобученной модели (современный способ — weights)
model = models.resnet18(weights="DEFAULT")  # или weights="IMAGENET1K_V1"
# Старый способ (deprecated, но ещё работает с warning):
# model = models.resnet18(pretrained=True)

model.eval()  # обязательно для inference!
# Если есть GPU: model = model.to("cuda")

# 2. Трансформации (точно те же, что использовались при обучении на ImageNet)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# 3. Загрузка примера картинки (можно заменить на свой путь)
url = "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=800"  # пример: котик
response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("RGB")

# или с диска:
# img = Image.open("your_image.jpg").convert("RGB")

# 4. Подготовка изображения
input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0)  # добавляем batch dimension: [1, 3, 224, 224]

# Если GPU: input_batch = input_batch.to("cuda")

# 5. Inference
with torch.no_grad():  # экономим память, отключаем градиенты
    output = model(input_batch)

# 6. Получаем предсказание (класс с максимальной вероятностью)
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

print(f"Predicted class index: {predicted_class}")
print(f"Confidence: {probabilities[predicted_class]:.4f}")

Predicted class index: 285
Confidence: 0.5818


In [1]:
import torch
import torch.nn as nn


# 1. Простая модель
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


# Создаём и обучаем (или просто инициализируем)
model = SimpleNet()

# --------------- Сохранение только весов (самый популярный способ) ---------------
torch.save(model.state_dict(), "my_model_weights.pth")
# или с учётом устройства и оптимизатора:
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'epoch': epoch,
# }, "checkpoint.pth")

# --------------- Загрузка ---------------
# Способ А: та же архитектура уже существует
loaded_model = SimpleNet()  # Важно: сначала создаём модель той же структуры!
loaded_model.load_state_dict(torch.load("my_model_weights.pth"))

# Способ Б: если модель на GPU, а сохраняли на CPU (или наоборот)
loaded_model.load_state_dict(
    torch.load("my_model_weights.pth", map_location="cpu")  # или "cuda", "cuda:0"
)

# Теперь можно использовать
loaded_model.eval()  # для inference
# или loaded_model.train() для дальнейшего обучения

SimpleNet(
  (fc1): Linear(in_features=10, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)